In [19]:
import json
import os
import numpy as np
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import wcs
from astropy.table import Table
import time
import requests
import shutil # save img locally
import h5py
import random

In [20]:
#print table to view data
SGA = Table.read('SGA_MorphsOnly.fits')
SGA

<Table length=138026>
 SGA_ID    SGA_GALAXY            GALAXY         ...  FRACIN_Z  HUBBLE_TYPE
 int64      bytes16             bytes29         ...  float32     float64  
------- ---------------- ---------------------- ... ---------- -----------
 284583  SGA-2020 284583             PGC1753576 ... 0.99611634        -5.0
 202259  SGA-2020 202259             PGC1560599 ... 0.99347186        -5.0
1376538 SGA-2020 1376538             PGC1183324 ... 0.98853034        -5.0
 117296  SGA-2020 117296              PGC058096 ... 0.99878794        -5.0
1098679 SGA-2020 1098679             PGC1255990 ... 0.99724495        -5.0
 972626  SGA-2020 972626 2MASXJ09470264+2312121 ...  0.9870488        -5.0
1098672 SGA-2020 1098672             PGC2800969 ...  0.9937631        -5.0
1376529 SGA-2020 1376529              PGC032587 ... 0.99987155        -5.0
 202246  SGA-2020 202246 2MASXJ16185371+0948266 ...  0.9996761        -5.0
1376540 SGA-2020 1376540 2MASXJ12390804+1703456 ...  0.9920145        -5.0
    ...              ...                    ... ...        ...         ...
1188856 SGA-2020 1188856               UGC08001 ... 0.98003256         9.0
1349492 SGA-2020 1349492              PGC135760 ...  0.7753037         9.0
1276074 SGA-2020 1276074             PGC1425523 ... 0.96871394         9.0
1077735 SGA-2020 1077735             PGC1032499 ...  0.9984691         9.0
1040516 SGA-2020 1040516              PGC027864 ...  0.9979983         9.0
 718927  SGA-2020 718927              PGC051523 ... 0.99053127         9.0
 388722  SGA-2020 388722             PGC1446565 ...  0.9917236         9.0
 793629  SGA-2020 793629             PGC1862932 ...   0.999169         9.0
1041033 SGA-2020 1041033              PGC036274 ... 0.99042934         9.0
 758057  SGA-2020 758057             PGC1673871 ... 0.92356014         9.0

In [8]:
# i=0
# while i < len(SGA):
#     if SGA['RA'][i] <= 25 and SGA['RA'][i] >= 24.6:
#         print(SGA['RA'][i], SGA['DEC'][i], SGA['HUBBLE_TYPE'][i])
#     i+=1

In [21]:
#Command to retrieve url datas
# RETURNS JPEG IMAGES, FOR VIEWING ONLY
def get_jpgcut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    print(url)
    res = requests.get(url, stream = True)
    file_name = 'Images/Sample_Gal{}.jpg'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [5]:
#Command to retrieve url datas for FITS files
def get_fitscut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.fits?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    #print(url)
    res = requests.get(url, stream = True)
    file_name = 'LittleGalaxies/Gal{}.fits'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        #print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [41]:
# #create some pretty pictures
# i=0
# while i < 2:
#     get_jpgcut(SGA['SGA_ID'][i],SGA['RA_LEDA'][i],SGA['DEC_LEDA'][i])
#     time.sleep(1)
#     i+=1

-5.0


In [24]:
i=0
while i < 10:
    get_jpgcut(SGA['SGA_ID'][45468+i],SGA['RA'][45468+i],SGA['DEC'][45468+i])
    i+=1

https://www.legacysurvey.org/viewer/cutout.jpg?ra=305.4612651561515&dec=-49.70220275250485&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Sample_Gal1311416.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=21.304267304600053&dec=2.0668900007826974&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Sample_Gal223377.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=17.407416909927665&dec=-45.897618856337246&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Sample_Gal414555.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=86.9299855284297&dec=-25.915565502681222&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Sample_Gal628976.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=230.11171706624137&dec=28.88633515374129&%22/pix=0.25&layer=ls-dr9&size=152
Image sucessfully Downloaded:  Images/Sample_Gal201888.jpg
https://www.legacysurvey.org/viewer/cutout.jpg?ra=349.99998959440865&de

In [43]:
f = h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/230offcuts.h5','a')

In [44]:
count = 5500
pixel = 230
brickid = f.create_dataset('brickid',(count,), maxshape=(None,))
flux_g = f.create_dataset('flux_g',(count,), maxshape=(None,))       #needed
flux_r = f.create_dataset('flux_r',(count,), maxshape=(None,))       #needed
flux_z = f.create_dataset('flux_z',(count,), maxshape=(None,))       #needed
fiberflux_g = f.create_dataset('fiberflux_g',(count,), maxshape=(None,))
fiberflux_r = f.create_dataset('fiberflux_r',(count,), maxshape=(None,))
fiberflux_z = f.create_dataset('fiberflux_z',(count,), maxshape=(None,))
psfsize_g = f.create_dataset('psfsize_g',(count,), maxshape=(None,))       #needed
psfsize_r = f.create_dataset('psfsize_r',(count,), maxshape=(None,))       #needed
psfsize_z = f.create_dataset('psfsize_z',(count,), maxshape=(None,))       #needed
psfdepth_g = f.create_dataset('psfdepth_g',(count,), maxshape=(None,))       #needed
psfdepth_r = f.create_dataset('psfdepth_r',(count,), maxshape=(None,))       #needed
psfdepth_z = f.create_dataset('psfdepth_z',(count,), maxshape=(None,))       #needed
nobs_g = f.create_dataset('nobs_g',(count,), maxshape=(None,))
nobs_r = f.create_dataset('nobs_r',(count,), maxshape=(None,))
nobs_z = f.create_dataset('nobs_z',(count,), maxshape=(None,))
fracin_g = f.create_dataset('fracin_g',(count,), maxshape=(None,))
fracin_r = f.create_dataset('fracin_r',(count,), maxshape=(None,))
fracin_z = f.create_dataset('fracin_z',(count,), maxshape=(None,))
release = f.create_dataset('release',(count,), maxshape=(None,))
objid = f.create_dataset('objid',(count,), maxshape=(None,))
source_type = f.create_dataset('source_type',(count,), maxshape=(None,))
maskbits = f.create_dataset('maskbits',(count,), maxshape=(None,))
ebv = f.create_dataset('ebv',(count,), maxshape=(None,))       #needed
images = f.create_dataset('images',(count,3,pixel,pixel), maxshape=(None, 3, pixel, pixel))       #needed
ra = f.create_dataset('ra',(count,), maxshape=(None,))       #needed
dec = f.create_dataset('dec',(count,), maxshape=(None,))       #needed
mag_g = f.create_dataset('mag_g',(count,), maxshape=(None,))
mag_r = f.create_dataset('mag_r',(count,), maxshape=(None,))
mag_z = f.create_dataset('mag_z',(count,), maxshape=(None,))
morph = f.create_dataset('morphology',(count,), maxshape=(None,))
#PROBLEMATIC ONE:
z_phot_median = f.create_dataset('z_phot_median',(count,), maxshape=(None,))

In [45]:
count_per_type = int((count/11))
print(count_per_type)

500


In [46]:
#collect data for (MORPHTYPE= -5)
i=0 #controls location within SGA catalog
j=0 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (count_per_type/2):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)

    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    
    #time.sleep(0.1) #0.1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [47]:
#collect data for (MORPHTYPE = -5) OFFSET
i=0 #controls location within SGA catalog
j=250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < 250:
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [48]:
#collect data for (MORPHTYPE = -2)
i=45468 #controls location within SGA catalog
j=(count_per_type*1) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (45468+int(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [49]:
#collect data for (MORPHTYPE = -2)
i=45468 #controls location within SGA catalog
j=(count_per_type*1)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (45468+int(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [50]:
#collect data for (MORPHTYPE = 0)
i=50217 #controls location within SGA catalog
j=(count_per_type*2) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (50217+(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [51]:
#collect data for (MORPHTYPE = 0) OFFSET
i=50217 #controls location within SGA catalog
j=(count_per_type*2)+250 #controls location within the h5 file, separate so you can skip around in catalog to make sure all morphologies are represented
while i < (50217+(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [52]:
#collect data for (MORPHTYPE = 1)
i=64744 #controls location within SGA catalog
j=(count_per_type*3) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (64744+int(count_per_type/2)):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
 #   time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [53]:
#collect data for (MORPHTYPE = 1)
i=64744 #controls location within SGA catalog
j=(count_per_type*3)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (64744+int(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
 #   time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [54]:
#collect data for (MORPHTYPE = 2)
i=67631 #controls location within SGA catalog
j=(count_per_type*4) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (67631+(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [55]:
#collect data for (MORPHTYPE = 2) OFFSET
i=67631 #controls location within SGA catalog
j=(count_per_type*4)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (67631+(count_per_type/2)):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [56]:
#collect data for (MORPHTYPE = 3)
i=73621 #controls location within SGA catalog
j=(count_per_type*5) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (73621+(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    i+=1
    j+=1

In [57]:
#collect data for (MORPHTYPE = 3)
i=73621 #controls location within SGA catalog
j=(count_per_type*5)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (73621+(count_per_type/2)):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    i+=1
    j+=1

In [58]:
#collect data for (MORPHTYPE = 4)
i=84697 #controls location within SGA catalog
j=(count_per_type*6) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (84697+int(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    i+=1
    j+=1

In [59]:
#collect data for (MORPHTYPE = 4) OFFSET
i=84697 #controls location within SGA catalog
j=(count_per_type*6)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (84697+(count_per_type/2)):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    i+=1
    j+=1

In [60]:
#collect data for (MORPHTYPE =5)
i=101991 #controls location within SGA catalog
j=(count_per_type*7) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (101991+int(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    i+=1
    j+=1

In [61]:
#collect data for (MORPHTYPE =5)
i=101991 #controls location within SGA catalog
j=(count_per_type*7)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (101991+int(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    i+=1
    j+=1

In [62]:
#collect data for (MORPHTYPE = 6)
i=130446 #controls location within SGA catalog
j=(count_per_type*8) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (130446+(count_per_type/2)):
   # get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [63]:
#collect data for (MORPHTYPE = 6) OFFSET
i=130446 #controls location within SGA catalog
j=(count_per_type*8)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (130446+(count_per_type/2)):
  #  get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
    time.sleep(0.1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [64]:
#collect data for (MORPHTYPE = 7)
i=133828 #controls location within SGA catalog
j=(count_per_type*9) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (133828+(count_per_type/2)):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
   # time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [65]:
#collect data for (MORPHTYPE = 7)
i=133828 #controls location within SGA catalog
j=(count_per_type*9)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (133828+(count_per_type/2)):
    #get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
   # time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [66]:
#collect data for (MORPHTYPE = 9)
i=136298 #controls location within SGA catalog
j=(count_per_type*10) #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (136298+(count_per_type/2)):
  #  get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i])
    img = fits.open('230FITS/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [67]:
#collect data for (MORPHTYPE = 9)
i=136298 #controls location within SGA catalog
j=(count_per_type*10)+250 #controls location within the h5 file, separate so you cacn skip around in catalog to make sure all morphologies are represented
while i < (136298+(count_per_type/2)):
  #  get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i]+round(random.uniform(-0.008, 0.008), 4),SGA['DEC'][i]+round(random.uniform(-0.008, 0.008), 4))
    img = fits.open('230offcuts/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    
   # time.sleep(1) #1 second pause per loop to not overwhelm server
    
    i+=1
    j+=1

In [68]:
f = h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/230offcuts.h5').close

In [4]:
#f = h5py.File('cutoutdata.hdf5','a')


In [6]:

# morph = f.create_dataset('morphology',(5500,), maxshape=(None,))
# i=0
#  while i < 500:
#     morph[i] = -5
#     i+=1
# while i < 1000:
#     # morph[i] = -2
#     i+=1
# while i < 1500:
#     morph[i] = 0
#     i+=1
# while i < 2000:
#     morph[i] = 1
#     i+=1
# while i < 2500:
#     morph[i] = 2
#     i+=1
# while i < 3000:
#     morph[i] = 3
#     i+=1
# while i < 3500:
#     morph[i] = 4
#     i+=1
# while i < 4000:
#     morph[i] = 5
#     i+=1
# while i < 4500:
#     morph[i] = 7
#     i+=1
# while i < 5000:
#     morph[i] = 9
#     i+=1


In [31]:
random_indices = random.sample(range(0, 138000), 10)
print(random_indices)

[71087, 56411, 57337, 118663, 114791, 131152, 70490, 69337, 107114, 69757]


In [34]:
j=0
for i in random_indices:
    get_fitscut(SGA['SGA_ID'][i],SGA['RA'][i],SGA['DEC'][i],100)
    img = fits.open('Cutouts_RandomSize/Gal{}.fits'.format(SGA['SGA_ID'][i]))
    images[j] = np.array(img[0].data)
    
    # Compile additional values into h5 file
    brickid[j] = SGA['BRICKID'][i]
    flux_g[j] = SGA['FLUX_G'][i]
    flux_r[j] = SGA['FLUX_R'][i]
    flux_z[j] = SGA['FLUX_Z'][i]
    fiberflux_g[j] = SGA['FIBERFLUX_G'][i]
    fiberflux_r[j] = SGA['FIBERFLUX_R'][i]
    fiberflux_z[j] = SGA['FIBERFLUX_R'][i]
    psfsize_g[j] = SGA['PSFSIZE_G'][i]
    psfsize_r[j] = SGA['PSFSIZE_R'][i]
    psfsize_z[j] = SGA['PSFSIZE_Z'][i] 
    psfdepth_g[j] = SGA['PSFDEPTH_G'][i] 
    psfdepth_r[j] = SGA['PSFDEPTH_R'][i]
    psfdepth_z[j] = SGA['PSFDEPTH_Z'][i] 
    nobs_g[j] = SGA['NOBS_G'][i]
    nobs_r[j] = SGA['NOBS_R'][i] 
    nobs_z[j] = SGA['NOBS_Z'][i] 
    fracin_g[j] = SGA['FRACIN_G'][i] 
    fracin_r[j] = SGA['FRACIN_R'][i] 
    fracin_z[j] = SGA['FRACIN_Z'][i]
    release[j] = SGA['RELEASE'][i]
    objid[j] = SGA['OBJID'][i]
    ebv[j] = SGA['EBV'][i]
    ra[j] = SGA['RA'][i]
    dec[j] = SGA['DEC'][i]
    mag_g[j] = SGA['G_MAG_SB26'][i]
    mag_r[j] = SGA['R_MAG_SB26'][i]
    mag_z[j] = SGA['Z_MAG_SB26'][i]
    morph[j] = SGA['HUBBLE_TYPE'][i]
    j+=1
    time.sleep(0.1) #1 second pause per loop to not overwhelm server


In [26]:
f = h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/offcuts.h5').close